In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Apr_weekday_MD

xi_list = GLS_Apr_weekday_MD.xi_list
P = GLS_Apr_weekday_MD.P
L = GLS_Apr_weekday_MD.L  # dimension of xi

size(P, 1), size(P, 2)

using JuMP
using Ipopt

mGLSJulia = Model(solver=IpoptSolver())

@variable(mGLSJulia, lam[1:size(P,1)] >= 0)

@variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @constraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @NLconstraint(mGLSJulia, sum(p[i,j] for j = 1:size(P,2)) == 1)
end

for l = 1:L
    @NLconstraint(mGLSJulia, sum(p[i,l] * lam[i] for i = 1:size(P,1)) == xi_list[l])
end
    
@NLobjective(mGLSJulia, Min, sum(p[1,j] for j = 1:size(P,2)))  # play no actual role, but could not use zero objective

solve(mGLSJulia)

getvalue(lam)

getobjectivevalue(mGLSJulia)

GLS_Apr_weekday_MD.saveDemandVec(getvalue(lam))

getvalue(p)

route_choice_matrix_Apr_MD = getvalue(p)

sum(route_choice_matrix_Apr_MD[1, :])

route_choice_matrix_dict_Apr_MD = Dict()

for row_idx = 1:size(route_choice_matrix_Apr_MD)[1]
    for col_idx = 1:size(route_choice_matrix_Apr_MD)[2]
        route_choice_matrix_dict_Apr_MD[(row_idx, col_idx)] = route_choice_matrix_Apr_MD[row_idx, col_idx]
    end
end

route_choice_matrix_dict_Apr_MD

outfile = open("../temp_files/route_choice_CDC_Apr_MD.json", "w")

JSON.print(outfile, route_choice_matrix_dict_Apr_MD)

close(outfile)

No dicts found; please check load_dicts...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    25199
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    18984

Total number of variables............................:     6384
                     variables with only lower bounds:     6384
                variables with lower and upper bounds:        0
                     variables with only 